In [1]:
import numpy as np
import pandas as pd
import wfdb
import ast
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
import os

from sklearn.utils import shuffle
import math
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,normalize, MinMaxScaler


import os
import wandb
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score
import warnings

In [2]:
### Preprocessing 
#   Using the super classes, multi label classification, excluding samples with no labels and considering atleast one label

path = 'ptb/'
Y = pd.read_csv(path+ 'ptbxl_database.csv', index_col = 'ecg_id')



data = np.array([wfdb.rdsamp(path+f)[0] for f in Y.filename_lr])
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))
    
agg_df = pd.read_csv(path+ 'scp_statements.csv', index_col = 0)

agg_df = agg_df[agg_df.diagnostic == 1]

def agg(y_dic):
    temp =[]
    
    for key in y_dic.keys():
        if key in agg_df.index:
            c = agg_df.loc[key].diagnostic_class
            if str(c) != 'nan':
                temp.append(c)
    return list(set(temp))

Y['diagnostic_superclass'] = Y.scp_codes.apply(agg)
Y['superdiagnostic_len'] = Y['diagnostic_superclass'].apply(lambda x: len(x))


#########

counts = pd.Series(np.concatenate(Y.diagnostic_superclass.values)).value_counts()

Y['diagnostic_superclass'] = Y['diagnostic_superclass'].apply(lambda x: list(set(x).intersection(set(counts.index.values))))

X_data = data[Y['superdiagnostic_len'] >= 1]
Y_data = Y[Y['superdiagnostic_len'] >= 1]

mlb = MultiLabelBinarizer()
mlb.fit(Y_data['diagnostic_superclass'])
y = mlb.transform(Y_data['diagnostic_superclass'].values)

########

## Stratify split

X_train = X_data[Y_data.strat_fold < 9]
y_train = y[Y_data.strat_fold < 9]

X_val = X_data[Y_data.strat_fold == 9]
y_val = y[Y_data.strat_fold == 9]

X_test = X_data[Y_data.strat_fold == 10]
y_test = y[Y_data.strat_fold == 10]

del X_data, Y_data, y



In [3]:
#########

# Standardizing

def apply_scaler(X, scaler):
    X_tmp = []
    for x in X:
        x_shape = x.shape
        X_tmp.append(scaler.transform(x.flatten()[:,np.newaxis]).reshape(x_shape))
    X_tmp = np.array(X_tmp)
    return X_tmp


scaler = StandardScaler()

scaler.fit(np.vstack(X_train).flatten()[:,np.newaxis].astype(float))

X_train_scale = apply_scaler(X_train, scaler)
X_test_scale = apply_scaler(X_test, scaler)
X_val_scale = apply_scaler(X_val, scaler)

del X_train, X_test, X_val

## Shuffling

X_train_scale, y_train = shuffle(X_train_scale, y_train, random_state = 42)

In [4]:
class DataGen(tf.keras.utils.Sequence):
    def __init__(self, X, y,batch_size = 16):
        self.batch_size = batch_size
        self.X = X
        self.y = y
        
    def __len__(self):
        return math.ceil(len(self.X) / self.batch_size)
    
    def __getitem__(self,idx):
        
        X_full = self.X[idx * self.batch_size:(idx + 1) *self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *self.batch_size]

            
        return np.transpose(X_full[..., np.newaxis], (0, 2, 1, 3)) ,batch_y
    
## Params

batch_size = 32
    
train_gen = DataGen(X_train_scale, y_train, batch_size = batch_size)
test_gen = DataGen(X_test_scale, y_test, batch_size = batch_size)

In [5]:
test = train_gen[0][0].shape
print(test)

(32, 12, 1000, 1)


In [6]:
import tensorflow.keras.backend as K

class attention(tf.keras.layers.Layer):
    
    def __init__(self, return_sequences = False, dim = 32, **kwargs):
        self.return_sequences = return_sequences
        self.dim = dim
        super(attention,self).__init__(**kwargs)
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1], self.dim),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1], self.dim),
                               initializer="zeros")
        self.V = self.add_weight(name = "Vatt", shape = (self.dim, 1), initializer = "normal")
        
        super(attention,self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x,self.W)+self.b)
        e = K.dot(e, self.V)
        a = K.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences :
            return output, a
        
        return K.sum(output, axis=1), a

    def get_config(self):
        base_config = super().get_config()
        config = {"return sequences" : tf.keras.initializers.serialize(self.return_sequences), "att dim" : tf.keras.initializers.serialize(self.dim)}
        return dict(list(base_config.items()) + list(config.items()))
    
## Resnet blocks

def relu_bn(inputs: tf.Tensor) -> tf.Tensor:
    
    
    dp = Dropout(0.5)(inputs)
    relu = ReLU()(dp)
    bn = BatchNormalization()(relu)
    return bn


def residual_block(x: tf.Tensor, downsample: bool, filters: int, kernel_size: int = 12) -> tf.Tensor:
    
    y = Conv1D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv1D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv1D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

In [7]:
## Params

sig_len = 1000
beat_size = 50

In [8]:
from tensorflow.keras.layers import Conv1D, Input, Attention, LSTM, Activation, Dense, Average,ReLU, BatchNormalization,Add, Reshape, Bidirectional, Concatenate

num_channel = 12
num_filters = 32
num_blocks_list = [2, 2, 2]

inputs = Input(shape = (num_channel, sig_len, 1), batch_size = None)

#### Beat Level 
x = K.reshape(inputs, (-1, beat_size,1 ))

x = Conv1D(32 ,12 ,padding = 'same')(x)
x = Activation('relu')(x)

for i in range(len(num_blocks_list)):
    num_blocks = num_blocks_list[i]
    for j in range(num_blocks):
        x = residual_block(x, downsample=(j==0 and i!=0), filters=num_filters)
    num_filters *= 2
    
x, _ = attention(name = "beat_att")(x)

##### Rhythm level
x = K.reshape(x,(-1, int(sig_len/beat_size) , 64))

x = Bidirectional(LSTM(32, return_sequences = True))(x)
x, _ = attention(name = "rhythm_att")(x)


#### Channel level

x = K.reshape(x, (-1, num_channel, 64))
x, _ = attention(name = "channel_att")(x)

outputs = Dense(5, activation = 'sigmoid')(x)


model = tf.keras.models.Model(inputs = inputs, outputs = outputs)

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy',tf.keras.metrics.AUC(multi_label = True)])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 12, 1000, 1) 0                                            
__________________________________________________________________________________________________
tf_op_layer_Reshape (TensorFlow [(None, 50, 1)]      0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 50, 16)       144         tf_op_layer_Reshape[0][0]        
__________________________________________________________________________________________________
activation (Activation)         (None, 50, 16)       0           conv1d[0][0]                     
_______________________________________________________________________________________

In [11]:
if not os.path.exists('3_level_att_saves'):
    os.mkdir('3_level_att_saves')

3840

In [ ]:
wandb.init(project = '3_level_att', name = 'run_1')

In [10]:
### Accuracy metric

def metrics(y_true, y_scores):
    y_pred = y_scores >= 0.5
    acc = np.zeros(y_pred.shape[-1])
    
    for i in range(y_pred.shape[-1]):
        acc[i] = accuracy_score(y_true[:,i], y_pred[:,i])
    return acc, np.mean(acc)

## Callback for logging and metrics 

class model_checkpoint(tf.keras.callbacks.Callback):

    def __init__(self, filepath, gen, monitor='loss',  options=None, **kwargs):

        super().__init__()

        self.filepath = filepath
        self.monitor = monitor
        self.test_data = gen
        
        
    def on_epoch_end(self, epoch, logs = {}) :
        
        test_len = len(self.test_data)
        score = []
        gt =[]

        for i in range(test_len):
            X,y = self.test_data[i][0], self.test_data[i][1]
            temp_score = self.model.predict(X)
            score.append(temp_score)
            gt.append(y)

        score = np.concatenate(score, axis = 0)
        gt = np.concatenate(gt, axis = 0)
        
        roc_auc = roc_auc_score(gt, score, average = 'macro')
        _, accuracy = metrics(gt, score)
        
        temp_path = f"{epoch+1}_roc_{roc_auc:.4f}.h5"
        path = os.path.join(self.filepath, temp_path)
        
        if epoch > 5 :
            self.model.save_weights(path)

        wandb.log({'train_loss' : logs['loss'], 'epoch' : epoch})
        wandb.log({'train_keras_auroc' : logs.get(self.monitor), 'epoch' : epoch})
        
        wandb.log({'test_loss' : logs['val_loss'], 'epoch' : epoch})
        wandb.log({'test_keras_auroc' : logs['val_auc'], 'epoch' : epoch})

        wandb.log({'test_roc_score' : roc_auc, 'epoch' : epoch})
        wandb.log({'test_accuracy_score' : accuracy, 'epoch' : epoch})
        
        logs['val_roc_auc'] = roc_auc
        logs['val_accuracy_score'] = accuracy
    
    def set_model(self, model):
        self.model = model

        
metric = 'auc'
checkpoint_filepath = '3_level_att_saves'

checkpoint = model_checkpoint(checkpoint_filepath, monitor = metric, gen = test_gen )


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        factor=0.1,
        patience=10,
        min_lr=0.001 * 0.001)

callbacks = [checkpoint, reduce_lr]     

In [1]:
history = model.fit(train_gen, epochs = 60, validation_data = test_gen, workers = 5 )

In [13]:
path_weights = r'49_roc_0.9216.h5'

model.load_weights(path_weights)

In [14]:
test_gen = DataGen(X_test_scale, y_test, batch_size = len(y_test))

pred = model.predict(test_gen[0][0])

roc_auc_score(y_test, pred, average='macro')

0.921606040850872

In [15]:
### Accuracy metric

def metrics(y_true, y_scores):
    y_pred = y_scores >= 0.5
    acc = np.zeros(y_pred.shape[-1])
    
    for i in range(y_pred.shape[-1]):
        acc[i] = accuracy_score(y_true[:,i], y_pred[:,i])
    return acc, np.mean(acc)

acc, mean_acc = metrics(y_test, pred)
print(f'class wise accuracy: {acc}')
print(f'accuracy: {mean_acc}')

class wise accuracy: [0.88534443 0.91585761 0.87979658 0.87101248 0.89089228]
accuracy: 0.888580674988442


In [16]:
### Class wise AUC

roc_score = roc_auc_score(y_test, pred, average='macro')
print(f'roc_score : {roc_score}')

def AUC(y_true: np.ndarray, y_pred: np.ndarray, verbose=False) -> float:
    """Computes the macro-average AUC score.

    Args:
        y_true (np.ndarray): list of labels
        y_pred (np.ndarray): list of predicted probabilities

    Returns:
        float: macro-average AUC score.
    """
    aucs = []
    assert len(y_true.shape) == 2 and len(y_pred.shape) == 2, 'Predictions and labels must be 2D.'
    for col in range(y_true.shape[1]):
        try:
            aucs.append(roc_auc_score(y_true[:, col], y_pred[:, col]))
        except ValueError as e:
            if verbose:
                print(
                    f'Value error encountered for label {col}, likely due to using mixup or '
                    f'lack of full label presence. Setting AUC to accuracy. '
                    f'Original error was: {str(e)}.'
                )
            aucs.append((y_pred == y_true).sum() / len(y_pred))
    return np.array(aucs)

class_auc = AUC(y_test, pred)
print(f'class wise AUC : {class_auc}')



roc_score : 0.921606040850872
class wise AUC : [0.91964736 0.88341405 0.92447519 0.94504497 0.93544863]


In [17]:
pred_values = pred >= 0.5

report = classification_report(y_test, pred_values, target_names = mlb.classes_)
print(report)

              precision    recall  f1-score   support

          CD       0.82      0.64      0.72       498
         HYP       0.78      0.43      0.56       263
          MI       0.83      0.67      0.74       553
        NORM       0.83      0.90      0.86       964
        STTC       0.81      0.72      0.76       523

   micro avg       0.82      0.73      0.77      2801
   macro avg       0.81      0.67      0.73      2801
weighted avg       0.82      0.73      0.76      2801
 samples avg       0.79      0.76      0.76      2801

C:\Users\likit\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
def multi_threshold_precision_recall(y_true: np.ndarray, y_pred: np.ndarray, thresholds: np.ndarray) :
    
    # Expand analysis to number of thresholds
    y_pred_bin = np.repeat(y_pred[None, :, :], len(thresholds), axis=0) >= thresholds[:, None, None]
    y_true_bin = np.repeat(y_true[None, :, :], len(thresholds), axis=0)

    # Compute true positives
    TP = np.sum(np.logical_and(y_true, y_pred_bin), axis=2)

    # Compute macro-average precision handling all warnings
    with np.errstate(divide='ignore', invalid='ignore'):
        den = np.sum(y_pred_bin, axis=2)
        precision = TP / den
        precision[den == 0] = np.nan
        with warnings.catch_warnings():  # for nan slices
            warnings.simplefilter("ignore", category=RuntimeWarning)
            av_precision = np.nanmean(precision, axis=1)

    # Compute macro-average recall
    recall = TP / np.sum(y_true_bin, axis=2)
    av_recall = np.mean(recall, axis=1)

    return av_precision, av_recall


def metric_summary(y_true: np.ndarray, y_pred: np.ndarray, num_thresholds: int = 10) :
    
    thresholds = np.arange(0.00, 1.01, 1. / (num_thresholds - 1), float)
    average_precisions, average_recalls = multi_threshold_precision_recall(
        y_true, y_pred, thresholds
    )
    f_scores = 2 * (average_precisions * average_recalls) / (average_precisions + average_recalls)
    auc = np.array(AUC(y_true, y_pred, verbose=True)).mean()
    return (
        f_scores[np.nanargmax(f_scores)],
        auc,
        f_scores,
        average_precisions,
        average_recalls,
        thresholds
    )

metric_summary(y_test, pred)


(0.8057840187406535,
 0.921606040850872,
 array([0.41142773, 0.76653767, 0.80528875, 0.80578402, 0.79801804,
        0.78467242, 0.75753944, 0.72340467, 0.64063727,        nan]),
 array([0.25899214, 0.65308984, 0.74734166, 0.78767813, 0.81797619,
        0.84193254, 0.85984238, 0.89507299, 0.92151608,        nan]),
 array([1.        , 0.92768531, 0.87297735, 0.82474187, 0.77901063,
        0.73470489, 0.67699183, 0.60698875, 0.49098474, 0.        ]),
 array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
        0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ]))